#MyJohnDeere API Best Practices#

This notebook contains hints, best practices, and errata to help use the MyJohnDeere API's.

*Note - This notebook requires that you be logged into Google.*

Copyright (c) 2019 Deere & Company
 
This software may be modified and distributed under the terms
of the MIT license.  See the LICENSE file for details.

Most MyJohnDeere API's use the same Accept header of application/vnd.deere.axiom.v3+json. We'll make that the default.

In [0]:
MYJOHNDEERE_V3_JSON_HEADERS = {'Accept': 'application/vnd.deere.axiom.v3+json',
               'Content-Type': 'application/vnd.deere.axiom.v3+json'}

#OAuth 1.0a Code for working with MyJohnDeere API#

Using the application you created on https://developer.deere.com, copy the "App Id" and "Shared Secret" and use those to set the `client_key` and `client_secret` variables

![Newly Created App](https://github.com/joshuajcarson/DevelopWithDeereOnboardingPictures/raw/master/NewlyCreatedApp.jpg)

In [0]:
client_key = 'place_client_key_here'
client_secret = 'place_client_secret_here'

#Leave the line below as-is. It's there as a reminder - if you did not enter your client key and secret, then the rest of this workbook will not work.
assert(client_key != 'place_client_key_here' and client_secret != 'place_client_secret_here'), "You need to update your client_key and client_secret in this cell"

First, you need to authenticate. You can find more details on the oAuth flow on [our developer site](https://developer.deere.com/#!help&doc=.%2Fgetstarted%2FHELPOAuth.htm&anchor=), and there are additional examples and an onboarding workbook in [github](https://github.com/johndeere/). 

The verification flow is similar to what John Deere customers will see when they grant access to your application. For this notebook, we're using a verifier code. In a real application, you may want to provide a callback url so that users do not have to copy and paste a verification code.

In [0]:
from requests_oauthlib import OAuth1Session
request_url = 'https://sandboxapi.deere.com/platform/oauth/request_token'
authorize_url = 'https://my.deere.com/consentToUseOfData'
access_token_url = 'https://sandboxapi.deere.com/platform/oauth/access_token'
def get_authorization_url(resource_owner_key):
  return authorize_url + '?oauth_token=' + resource_owner_key

def get_request_token_information_for_myjohndeere_api():
  oauth_request_token_step = OAuth1Session(client_key, client_secret=client_secret, callback_uri='oob')
  request_token_response = oauth_request_token_step.fetch_request_token(request_url)
  resource_owner_key = request_token_response.get('oauth_token')
  resource_owner_secret = request_token_response.get('oauth_token_secret')
  complete_authorize_url = get_authorization_url(resource_owner_key)
  return resource_owner_key, resource_owner_secret, complete_authorize_url 
request_resource_owner_key, request_resource_owner_secret, complete_authorize_url = get_request_token_information_for_myjohndeere_api()
"Click On " + complete_authorize_url + " and paste the results in the next cell as the verifier variable"

Follow the link from the results of the cell above, and copy the "Verifier" from that screen into the cell below
![Verifier Screen](https://github.com/joshuajcarson/DevelopWithDeereOnboardingPictures/raw/master/VerifierScreen.jpg)

In [0]:
verifier = 'place_verifier_here'

#Leave the line below as-is. It is a reminder, because you need the verifier in order to finish authenticating.
assert(verifier != 'place_verifier_here'), 'The verifier in this cell must be replaced by the verified that you should get from MyJohnDeere'

Using the `verifier` from the step above, we can exchange the oAuth request token for an access token. The access token is what you will use to authenticate all the other API calls.

In [0]:
def get_verifier_token_information_for_myjohndeere_api(resource_owner_key, resource_owner_secret, verifier):
  oauth_verifier_step = OAuth1Session(client_key,
                          client_secret=client_secret,
                          resource_owner_key=resource_owner_key,
                          resource_owner_secret=resource_owner_secret,
                          verifier=verifier)
  oauth_tokens = oauth_verifier_step.fetch_access_token(access_token_url)
  resource_owner_key = oauth_tokens.get('oauth_token')
  resource_owner_secret = oauth_tokens.get('oauth_token_secret')
  return resource_owner_key, resource_owner_secret

verifier_resource_owner_key, verifier_resource_owner_secret = get_verifier_token_information_for_myjohndeere_api(request_resource_owner_key, request_resource_owner_secret, verifier)
def get_oauth_session_for_myjohndeere_api(resource_owner_key, resource_owner_secret):
  return OAuth1Session(client_key, client_secret=client_secret, resource_owner_key=resource_owner_key, resource_owner_secret=resource_owner_secret)

oauth_session = get_oauth_session_for_myjohndeere_api(verifier_resource_owner_key, verifier_resource_owner_secret)

#Using `links` within the MyJohnDeere API#

The MyJohnDeere API uses [HATEOAS](https://en.wikipedia.org/wiki/HATEOAS) links. Let's explore by calling the `API Catalog` and following links from there.

In [0]:
api_catalog_url = 'https://sandboxapi.deere.com/platform/'

api_catalog_response = oauth_session.get(api_catalog_url, headers = MYJOHNDEERE_V3_JSON_HEADERS)

api_catalog_response.json()

Adding a helper method can make links easier to use. The following functions convert from the JSON link object into a python dictionary. Now the API Catalog is a little easier to work with.

In [0]:
def convert_links_array_to_dictionary(links_a):
    link_dict = dict()
    for link_o in links_a:
        key = link_o['rel']
        value = link_o['uri']
        link_dict[key] = value
    return link_dict
api_catalog_links_as_dictionary = convert_links_array_to_dictionary(api_catalog_response.json()['links'])
api_catalog_links_as_dictionary

def replace_links_as_object_array_with_links_as_dictionary(object_with_links_to_convert):
  object_with_links_converted = object_with_links_to_convert
  object_with_links_converted['links'] = convert_links_array_to_dictionary(object_with_links_to_convert['links'])
  return object_with_links_converted

api_catalog_with_clean_links = replace_links_as_object_array_with_links_as_dictionary(api_catalog_response.json())
api_catalog_with_clean_links

That looks easier to work with. The `rel` has been converted to an attribute and the `uri` is now the value. Let us take it a step farther and modify the object itself to make using the responses themselves a bit easier.

When calling the MyJohnDeere API for resources, you can control which links can appear in the response. This is done via the `showLinks` request param, where the values act as a filter to on the returned links. Let's look at the `API Catalog` again, but with fewer links.

*Extra tip:* Not every link will be present; links only appear for API endpoints that you have permission to use. If your user context changes or you switch organizations, the available links may change.

In [0]:
api_catalog_showLinks_request_param = {'showLinks': 'currentToken,currentUser,organizations'}

api_catalog_response_with_request_param = oauth_session.get(api_catalog_url, headers = MYJOHNDEERE_V3_JSON_HEADERS, params = api_catalog_showLinks_request_param)
api_catalog_response_with_request_param.json()

Using the `showLinks` request parameter is useful once you know which links your application needs. Fewer links will result in faster API responses.

Now lets chain our functions together to get just the links we want.

In [0]:
def create_show_links_request_param(links_to_show_comma_separated = None):
  return {'showLinks' : links_to_show_comma_separated}

def get_myjohndeere_api_json_response(oauth_session, myjohndeere_uri, headers = MYJOHNDEERE_V3_JSON_HEADERS, params = None):
  json_response = oauth_session.get(myjohndeere_uri, headers = headers, params = params).json()
  return replace_links_as_object_array_with_links_as_dictionary(json_response)

api_catalog_response_with_clean_links = get_myjohndeere_api_json_response(oauth_session, myjohndeere_uri = api_catalog_url, params = create_show_links_request_param('currentToken,currentUser,organizations'))
api_catalog_response_with_clean_links

Now that we can manage links, try making additional API calls by following the links. Let's follow the '`currentUser`' link to see more information about the currently logged in user.

In [0]:
current_user_link = api_catalog_response_with_clean_links['links']['currentUser']
current_user_response = get_myjohndeere_api_json_response(oauth_session, myjohndeere_uri = current_user_link)
current_user_response

#Working with the `Collection` object : counts, nextPage links, and combining results#

Many of the MyJohnDeere API's return collections of objects, as compared to the single object responses we've seen with the `API Catalog` and `currentUser` endpoints. These collections are wrapped in a JSON object that contains metadata to support pagination. You can see an example by following the '`organizations`' link from the `currentUser`

In [0]:
organizations_for_current_user_link = current_user_response['links']['organizations']
get_myjohndeere_api_json_response(oauth_session, myjohndeere_uri = organizations_for_current_user_link)

The collection wrapper contains links for '`self`', '`nextPage`', and '`previousPage`'. There is also a `total` attribute that represents how many objects exist within the collection across all pages, and a `values` attribute that contains the actual collection of objects. 


In [0]:
def get_myjohndeere_api_collection_json_response(oauth_session, myjohndeere_uri, headers = MYJOHNDEERE_V3_JSON_HEADERS, params = None):
  collection_json_response = get_myjohndeere_api_json_response(oauth_session, myjohndeere_uri, headers, params)
  values_from_collection = collection_json_response['values']
  values_to_add_back_to_collection = []
  for object in values_from_collection:
    values_to_add_back_to_collection.append(replace_links_as_object_array_with_links_as_dictionary(object))
  collection_json_response['values'] = values_to_add_back_to_collection
  return collection_json_response

In [0]:
organizations_for_current_user_link = current_user_response['links']['organizations']
get_myjohndeere_api_collection_json_response(oauth_session, myjohndeere_uri = organizations_for_current_user_link)

Now that we have simplified the JSON that we're working with, we will next handle pagination. The MyJohnDeere API uses [pagination](https://developer.deere.com/#!help&doc=.%2Fgetstarted%2FPaginationGuide.htm&anchor=) in order to reduce the load per API request. 

In order to control pagination, MyJohnDeere API's use matrix parameters `start` and `count`. To demonstrate, set the `count` to one.

In [0]:
organizations_for_current_user_link = current_user_response['links']['organizations']
organizations_for_current_user_link_with_count_of_1 = organizations_for_current_user_link + ';count=1'
get_myjohndeere_api_collection_json_response(oauth_session, myjohndeere_uri = organizations_for_current_user_link_with_count_of_1)

Assuming the current user is a member of two organizations, we now see a `nextPage` link which has `start=1` and `count=1`. Let's enhance our last method to follow the `nextPage` link.

In [0]:
def get_myjohndeere_api_collection_values_for_all_pages(oauth_session, myjohndeere_uri, headers = MYJOHNDEERE_V3_JSON_HEADERS, params = None):
  collection_response = get_myjohndeere_api_collection_json_response(oauth_session, myjohndeere_uri, headers, params)
  collection_links = collection_response['links']
  while 'nextPage' in collection_links:
    next_page_response = get_myjohndeere_api_collection_json_response(oauth_session, collection_links['nextPage'], headers, params)
    collection_links = next_page_response['links']
    collection_response['values'].extend(next_page_response['values'])
  return collection_response['values']

In [0]:
organizations_for_current_user_link = current_user_response['links']['organizations']
organizations_for_current_user_link_with_count_of_1 = organizations_for_current_user_link + ';count=1'
get_myjohndeere_api_collection_values_for_all_pages(oauth_session, myjohndeere_uri = organizations_for_current_user_link_with_count_of_1)

To be more realistic, add another helper function to set the pagination count to 100. This is the maximum page size.

In [0]:
def default_to_count_100(followable_link) :
  return followable_link + ";count=100"

To bring all of this together, get the organization list with a page size of 100. Add a `showLinks` request param to limit links to just the `machines`, `files`, and `fields` links. 

*Extra tip: * If you don't want pagination, you can disable it by adding a header named `No_Paging` with value of `true`. For some endpoints you can also disable links by adding a query param `showLinks=none`.

In [0]:
show_links_request_param_for_organization_list = {'showLinks' : 'machines,files,fields'}

organizations_for_current_user_link = current_user_response['links']['organizations']
organizations_for_current_user_link_with_page_size_of_100 = default_to_count_100(organizations_for_current_user_link)
get_myjohndeere_api_collection_values_for_all_pages(oauth_session, organizations_for_current_user_link_with_page_size_of_100, params = show_links_request_param_for_organization_list)

#Using `embed` to optimize data retrieval#

To replace multiple API calls to retrieve several related objects, the MyJohnDeere API's support the concept of an '`embed`'. Instead of retrieving a list of fields and then making a series of API calls to get the associated `client` and `farm` for each field, you can request that clients and farms are embedded in the initial fields call. Then each `field` in the fields list will have pre-populated `client` and `farm` attributes.

Let's start by finding an organization that has a `fields` link.

In [0]:
show_field_links_request_param_for_organization_list = {'showLinks' : 'fields'}

organization_list_for_logged_in_user = get_myjohndeere_api_collection_values_for_all_pages(oauth_session, organizations_for_current_user_link_with_page_size_of_100, params = show_field_links_request_param_for_organization_list)
organization_which_has_fields_link = None
for organization in organization_list_for_logged_in_user:
  if 'fields' in organization['links']:
    organization_which_has_fields_link = organization
    break

assert(organization_which_has_fields_link != None), "The logged in user is not a staff member of any organization in which they can see fields"
organization_which_has_fields_link

Let's follow the `fields` link to see which fields are available to our user within this organization.

In [0]:
organization_field_list_uri = organization_which_has_fields_link['links']['fields']
organization_field_list_uri_with_count_100 = default_to_count_100(organization_field_list_uri)


field_list_for_organization = get_myjohndeere_api_collection_values_for_all_pages(oauth_session, organization_field_list_uri_with_count_100)
field_list_for_organization

As long as your organization has at least one field, you should see the `field` object with a variety of links. We could follow the links to retrieve the associated `client`, `farm`, and `boundary`, but it's faster to get everything in one API call. 

Let's make the same call as before, but use the `embed` request param to get more information.

In [0]:
farm_client_activeBoundary_embed_request_param = {'embed' : 'farms,clients,activeBoundary'}

field_list_for_organization_with_embed = get_myjohndeere_api_collection_values_for_all_pages(oauth_session, organization_field_list_uri_with_count_100, params = farm_client_activeBoundary_embed_request_param)
field_list_for_organization_with_embed

We strongly encourage our Connected Software Companies to use `embed` on frequently-used API requests. If you find an API that you wish had an `embed`, please let John Deere API support know.

#Using Deere ETags to monitor for changes#

[Deere ETags](https://developer.deere.com/#!help&doc=.%2Fgetstarted%2FHELPdeereTags.htm&anchor=) is a custom implementation of ETags. It helps keep data in sync between your system and ours by telling you what changed since the last time you made an API call.


In [0]:
def create_myjohndeere_api_header_with_deere_etag(deere_etag_value = 'nil'):
  return {'Accept': 'application/vnd.deere.axiom.v3+json', 'x-deere-signature' : deere_etag_value}

In [0]:
deere_etag_value = 'nil'

json_plus_deere_etag_headers = create_myjohndeere_api_header_with_deere_etag(deere_etag_value)

field_list_response_with_deere_etag = oauth_session.get(organization_field_list_uri_with_count_100, 
                                                        params = farm_client_activeBoundary_embed_request_param,
                                                        headers = json_plus_deere_etag_headers)
field_list_response_with_deere_etag.json()

The response looks the same, except for the new `x-deere-signature` header. It contains a UUID that can be used next time we make this API call.

In [0]:
field_list_response_with_deere_etag.headers['x-deere-signature']

We can use this UUID in future requests to see if anything has changed. Let's copy the `x-deere-signature` guid into a new request. 

*Extra tip:* This UUID is only valid for a single URL. For each combination of path parameters, you'll need to store a separate ETag UUID.

In [0]:
header_with_deere_etag_value = create_myjohndeere_api_header_with_deere_etag(field_list_response_with_deere_etag.headers['x-deere-signature'])

In [0]:
field_list_response_with_deere_etag_using_etag = oauth_session.get(organization_field_list_uri_with_count_100, 
                                                        params = farm_client_activeBoundary_embed_request_param,
                                                        headers = header_with_deere_etag_value)
field_list_response_with_deere_etag_using_etag

In [0]:
if field_list_response_with_deere_etag_using_etag.status_code == 200:
  #If a 200 response is seen, your own application would need to update your downloaded field list. This statement instead just updates the x-deere-signature to a new value
  header_with_deere_etag_value = create_myjohndeere_api_header_with_deere_etag(field_list_response_with_deere_etag_using_etag.headers['x-deere-signature'])

#Optimizing your calls with GZip#

You can reduce network traffic by asking the API's to compress their responses. The MyJohnDeere API's support GZip compression.

Let's add an `Accept-Encoding=GZip` header and make some more API calls.

In [0]:
MYJOHNDEERE_V3_JSON_AND_GZIP_HEADERS = {'Accept': 'application/vnd.deere.axiom.v3+json', 'Content-Type': 'application/vnd.deere.axiom.v3+json', 'Accept-Encoding' : 'gzip' }

organization_field_list_with_embed_responses_gzip = oauth_session.get(organization_field_list_uri_with_count_100, headers = MYJOHNDEERE_V3_JSON_AND_GZIP_HEADERS, params = farm_client_activeBoundary_embed_request_param)
field_list_response_size_in_byte_with_gzip = len(organization_field_list_with_embed_responses_gzip.content)
print("Response size of of the field list with GZip was " + str(field_list_response_size_in_byte_with_gzip))

#Recommended administration features to add to your integration#

###Using the `currentToken` API to avoid authentication errors###

Unlike oAuth 2, our oAuth 1 access tokens cannot be refreshed. Access tokens for the MyJohnDeere API's last for one year, and once they expire they cannot be reused. Remind your customers to renew their tokens before the year is finished. We'll use the 'currentToken' link from the API Catalog to see when our token expires.

In [0]:
current_token_uri = api_catalog_response_with_clean_links['links']['currentToken']
current_token_json_response = get_myjohndeere_api_json_response(oauth_session, current_token_uri)
current_token_json_response

In [0]:
print("Your token will expire on " + current_token_json_response['expirationDate'])

We recommend that you alert your customers within your own application when tokens are about to expire - and give them plenty of advance notice.

###Using the `currentUser` object as a 'foreign key' into the MyJohnDeere API###

Remember that user accounts in your system may be set up differently than user accounts in the John Deere system. Especially with large farming organizations, it is useful to be able to understand who a user is within John Deere Operations Center. We'll use the 'currentUser' link from the API Catalog to discover some key details about the logged in user.

In [0]:
current_user_uri = api_catalog_response_with_clean_links['links']['currentUser']
current_user_json_response = get_myjohndeere_api_json_response(oauth_session, current_user_uri)
current_user_json_response

The `currentUser` object contains a few key elements that you'll likely find useful. The `userType` attribute lets you know if the logged in user is a John Deere customer or a John Deere dealer, which you can use to give a more customized experience. The `accountName` attribute is the username of the customer within the John Deere system. Your own tooling should keep a record of this whenever you encounter errors that you plan to submit to John Deere's [API support team](mailto:APIDevSupport@JohnDeere.com). This will help the support team help you fix issues quickly.

#Handling API errors#

###Dealing with 500 responses###

While we target high uptime, every web API has problems sometimes. We've documented [how to handle some of the most common errors](https://developer.deere.com/#!help&doc=.%2Fgetstarted%2FHELP429.htm&anchor=).

When you do encounter an error, please don't retry immediately - that makes things worse! We recommend that you use an exponential backoff loop. For this code section, we'll use [httpstat.us](https://httpstat.us/) as a demonstration tool to automatically generate 500 errors.

*Extra tip:* This demo simply doubles the wait time after each failure. Another good alternative (in milliseconds) is `Thread.Sleep((int) Math.Pow(10, retryCount));`.

In [0]:
uri_that_always_generates_500_errors = "https://httpstat.us/500"
oauth_session.get(uri_that_always_generates_500_errors)

Now we'll add some code to retry when we encounter a 500. We will also import some libraries for dealing with time.

In [0]:
import time
import datetime

def is_status_code_a_server_error(http_status_code):
  return http_status_code >= 500 and http_status_code < 600

def print_current_time_to_console():
  print("Making a call at " + str(datetime.datetime.now()))

def call_api_with_500_retry_logic(oauth_session, uri_to_call):
  print_current_time_to_console()
  timer_in_seconds = 1
  api_response = oauth_session.get(uri_to_call)
  while(is_status_code_a_server_error(api_response.status_code) and timer_in_seconds < 30):
    time.sleep(timer_in_seconds)
    timer_in_seconds = timer_in_seconds * 2
    print_current_time_to_console()
    api_response = oauth_session.get(uri_to_call)
  print("Unable to succeed in the API call. This would be a good point to log the exact API call made, the username of the caller, as well as the date")

In [0]:
call_api_with_500_retry_logic(oauth_session, uri_that_always_generates_500_errors)

###Dealing with 429 responses###

The MyJohnDeere API has [built-in flood protection](https://developer.deere.com/#!documentation&doc=myjohndeere%2F429.htm&anchor=) that generates HTTP 429 responses. It is triggered by too many requests from a single application, or when the MyJohnDeere API server is under unusually high load.

As a demonstration, we'll set up a `mock` to simulate `429` responses from the `API Catalog`.



In [0]:
!pip install requests_mock

In [0]:
import requests
import requests_mock

mock_api_catalog_uri = 'mock://sandboxapi.deere.com/platform/'

session = requests.Session()
adapter = requests_mock.Adapter()
session.mount('mock', adapter)

retry_after_two_seconds_header = {'Retry-After' : '2'}
adapter.register_uri('GET', mock_api_catalog_uri, status_code = 429, headers = retry_after_two_seconds_header)

With the 429 response, the MyJohnDeere API server recommendations how long to wait - in seconds - before attempting to retry. We will use that response header in order to setup our sleep timers.

In [0]:
import time
import datetime

def is_status_code_429(http_status_code):
  return http_status_code == 429

def print_current_time_to_console():
  print("Making a call at " + str(datetime.datetime.now()))
  
def call_api_with_429_retry_logic(session, uri_to_call):
  print_current_time_to_console()
  attempts = 0
  api_response = session.get(uri_to_call)
  while(is_status_code_429(api_response.status_code) and attempts < 3):
    time.sleep(int(api_response.headers['Retry-After']))
    attempts = attempts + 1
    print_current_time_to_console()
    api_response = session.get(uri_to_call)
  print("For the 429, it can still be useful to have a maximum amount of retrys before considering the API call a failure")

In [0]:
call_api_with_429_retry_logic(session, mock_api_catalog_uri)

#Selecting the right organization for a grower#

For most applications connected to the MyJohnDeere API, your end user is a farmer who is unlikely to be a member of multiple organizations in Operations Center. If a user is a member of multiple organizations, you will need to select which one to use. In the second cell, make sure to change the `selected_organization_index` to an organization from the results from the first cell.

In [0]:
current_user_response = get_myjohndeere_api_json_response(oauth_session, current_user_uri)
organizations_for_current_user_uri = current_user_response['links']['organizations']
organizations_for_current_user_uri = default_to_count_100(organizations_for_current_user_uri)
organizations_for_current_user = get_myjohndeere_api_collection_values_for_all_pages(oauth_session, organizations_for_current_user_uri)
organization_index = 0
for organization in organizations_for_current_user:
  print("[" + str(organization_index) + "] " + organization['name'])
  organization_index = organization_index + 1

In [0]:
#Replace the value of selected_organization_index to one of the values listed from the cell above
selected_organization_index = 0

selected_organization = organizations_for_current_user[selected_organization_index]
print("The organization chosen has a name of \"" + selected_organization['name'] + "\" and an id of " + selected_organization['id'])

#Selecting the right organization for a contractor or a John Deere dealer#

Applications that target contractors or John Deere dealers often need to manage organizations differently. It is common for a contractor or dealer to support a large number of organizations, and your application will need a way to let the user select which one to use.

Note that in this case you want to keep data separated by organization. Merging data across organizations is probably not what a dealer or contractor wants.

In [0]:
api_catalog_response = get_myjohndeere_api_json_response(oauth_session, api_catalog_url)
organizations_uri = api_catalog_response['links']['organizations']
organizations_uri = default_to_count_100(organizations_uri)

organizations_visible_to_logged_in_user = get_myjohndeere_api_collection_values_for_all_pages(oauth_session, organizations_uri)
organization_index = 0
for organization in organizations_visible_to_logged_in_user:
  print("[" + str(organization_index) + "] " + organization['name'])
  organization_index = organization_index + 1

In [0]:
#Replace the values in the array below with the organizations you want to pick from above
selected_organizations_array = [0, 1]

selected_organizations = [organizations_visible_to_logged_in_user[i] for i in selected_organizations_array]
print("The number of organizations chosen was " + str(len(selected_organizations)))

#Getting your application to production#

###Application access while in Sandbox###

Applications on developer.deere.com have two lifecycle phases. In "Sandbox", your application is only available to your application team. In "Production", real customers can use your application. This is the only major difference between the two - in both phases, your application calls the production MyJohnDeere backend.

This is a picture showing the "Team Members" of an application on developer.deere.com. Only "Team" members of a `Sandbox` application on developer.deere.com are allowed to authenticate. You can invite additional team members as needed for development and testing.

![Team Section on developer.deere.com](https://github.com/JohnDeere/DevelopWithDeereNotebooks/raw/master/Best%20Practices/TeamOnDeveloperDeereCom.png)

###Getting to production###

When your application is ready to go to production with the MyJohnDeere API, it must pass two checks before we will grant Production access.

Schedule a review with John Deere's API support team. You do not need to share your source code - we just want to check our logs. Your application should be used like a real customer would use your application. This can include initial signup, creating data, and polling the MyJohnDeere API's.

If anything critical is noticed in the review, such as potential DDoS attacks or inefficient API usage, we'll make suggestions.

The John Deere Business Development team needs a signed API usage agreement. This agreement goes over all of the APIs required, the usage limits for those APIs, and allowed behaviors.

Once both steps are complete, production access will be granted. At that time, change the base URI you use to call MyJohnDeere from 'https://sandboxapi.deere.com/platform/' to 'https://partnerapi.deere.com/platform/'.